In [4]:
import pandas as pd
import csv

In [5]:

# List of CSV files to inspect and clean
csv_files = ["201312-citibike-tripdata.csv", "201311-citibike-tripdata.csv", "201310-citibike-tripdata.csv"]

# Function to clean individual dataframes
def clean_data(df):
    # Convert starttime and stoptime to datetime
    df['starttime'] = pd.to_datetime(df['starttime'], errors='coerce')
    df['stoptime'] = pd.to_datetime(df['stoptime'], errors='coerce')
    
    # Drop rows with NaT in starttime or stoptime
    df.dropna(subset=['starttime', 'stoptime'], inplace=True)
    
    # Check for missing values and fill or drop them
    df['gender'].fillna(0, inplace=True)
    
    # Ensure correct data types for numeric fields
    df['tripduration'] = pd.to_numeric(df['tripduration'], errors='coerce')
    df['start station id'] = pd.to_numeric(df['start station id'], errors='coerce')
    df['end station id'] = pd.to_numeric(df['end station id'], errors='coerce')
    df['bikeid'] = pd.to_numeric(df['bikeid'], errors='coerce')
    df['birth year'] = pd.to_numeric(df['birth year'], errors='coerce')
    df['gender'] = pd.to_numeric(df['gender'], errors='coerce')
    
    # Drop rows with invalid numeric values
    df.dropna(subset=['tripduration', 'start station id', 'end station id', 'bikeid', 'birth year', 'gender'], inplace=True)
    
    return df

# Combine and clean all CSV files
combined_df = pd.concat([clean_data(pd.read_csv(file)) for file in csv_files])

# Save the combined and cleaned DataFrame to a new CSV file
cleaned_file = "combined_citibike_data_cleaned.csv"
combined_df.to_csv(cleaned_file, index=False)

print("Combined and cleaned CSV files successfully!")
print(f"Cleaned data saved to {cleaned_file}")


Combined and cleaned CSV files successfully!
Cleaned data saved to combined_citibike_data_cleaned.csv


In [6]:

# Load your dataset
data = pd.read_csv('combined_citibike_data_cleaned.csv')

# Define the lookup table for NYC zip codes
nyc_zip_codes = [
    {"zip_code": "10001", "lat_min": 40.7440, "lat_max": 40.7506, "lon_min": -74.0060, "lon_max": -73.9940},
    {"zip_code": "10002", "lat_min": 40.7153, "lat_max": 40.7234, "lon_min": -73.9890, "lon_max": -73.9780},
    {"zip_code": "10003", "lat_min": 40.7281, "lat_max": 40.7361, "lon_min": -73.9950, "lon_max": -73.9840},
    # Add more zip codes and their lat/lon ranges
]

# Function to get zip code from lat/lon using the lookup table
def get_zip_code_from_lookup(lat, lon, lookup_table):
    for entry in lookup_table:
        if entry["lat_min"] <= lat <= entry["lat_max"] and entry["lon_min"] <= lon <= entry["lon_max"]:
            return entry["zip_code"]
    return None

# Apply function to get zip codes for start locations
data['start_station_zip_code'] = data.apply(lambda row: get_zip_code_from_lookup(row['start station latitude'], row['start station longitude'], nyc_zip_codes), axis=1)

# Apply function to get zip codes for end locations
data['end_station_zip_code'] = data.apply(lambda row: get_zip_code_from_lookup(row['end station latitude'], row['end station longitude'], nyc_zip_codes), axis=1)

# Save the updated dataset with zip codes
data.to_csv('zip_codes.csv', index=False)
